In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras as keras
import matplotlib.pyplot as plt

In [ ]:
tf.__version__

'2.8.0'

In [ ]:
digits = pd.read_csv('sample_data/mnist_train_small.csv')

In [ ]:
X = digits.iloc[:, 1:]
y = digits.iloc[:, 0]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler
X_train = X_train.astype(np.float64)
X_test = X_test.astype(np.float64)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
img_rows, img_cols = 28, 28

X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)

In [ ]:
X_train.shape

(18999, 28, 28, 1)

In [ ]:
X_test.shape

(1000, 28, 28, 1)

In [ ]:
y_train_cat = keras.utils.to_categorical(y_train, num_classes=10)
y_test_cat = keras.utils.to_categorical(y_test, num_classes=10)
y_train_cat[:5]

array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

In [ ]:
y_train_cat = y_train_cat.reshape((-1, 1,1,10))

In [ ]:
y_train_cat.shape

(18999, 1, 1, 10)

In [ ]:
y_train_cat[:2]

array([[[[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]]],


       [[[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]]]], dtype=float32)

In [ ]:
y_test_cat = y_test_cat.reshape((-1, 1,1,10))

In [ ]:
cnn_model = keras.Sequential()

cnn_model.add(keras.layers.Conv2D(input_shape=(28, 28, 1), activation=tf.nn.relu, filters=3, \
                     kernel_size=3, strides=1, padding='valid'))

cnn_model.add(keras.layers.Conv2D(activation=tf.nn.relu, filters=6, \
                     kernel_size=5, strides=1, padding='valid'))

cnn_model.add(keras.layers.Conv2D(activation=tf.nn.relu, filters=12, \
                     kernel_size=7, strides=1, padding='valid'))

cnn_model.add(keras.layers.Conv2D(activation=tf.nn.relu, filters=24, \
                     kernel_size=9, strides=1, padding='valid'))

cnn_model.add(keras.layers.Conv2D(activation=tf.nn.relu, filters=216, kernel_size=8, strides=1))

cnn_model.add(keras.layers.Conv2D(activation=tf.nn.relu, filters=108, kernel_size=1, strides=1))

cnn_model.add(keras.layers.Conv2D(activation='softmax', filters=10, kernel_size=1, strides=1))

In [ ]:
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 3)         30        
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 6)         456       
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 12)        3540      
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 24)          23352     
                                                                 
 conv2d_4 (Conv2D)           (None, 1, 1, 216)         331992    
                                                                 
 conv2d_5 (Conv2D)           (None, 1, 1, 108)         23436     
                                                                 
 conv2d_6 (Conv2D)           (None, 1, 1, 10)          1

In [ ]:
BATCH_SIZE = 16

optimizer = keras.optimizers.Adam(lr=0.0001)

# look for monitor values at https://keras.io/callbacks/
#learning_rate_reduction = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, min_lr=0.00001)

data_augmentor = keras.preprocessing.image.ImageDataGenerator(
            rotation_range=10,
            zoom_range=0.1,
            width_shift_range=0.1,
            height_shift_range=0.1)

# propar values for loss are present at https://keras.io/losses/
cnn_model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = cnn_model.fit(data_augmentor.flow(X_train, y_train_cat, batch_size=BATCH_SIZE), epochs=10,
                                    validation_data=(X_test, y_test_cat),
                                    steps_per_epoch=X_train.shape[0]//BATCH_SIZE)

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


1187/1187 [==============================] - 39s 32ms/step - loss: 0.9281 - accuracy: 0.7034 - val_loss: 0.2603 - val_accuracy: 0.9150
Epoch 2/10
1187/1187 [==============================] - 38s 32ms/step - loss: 0.3786 - accuracy: 0.8846 - val_loss: 0.1599 - val_accuracy: 0.9520
Epoch 3/10
1187/1187 [==============================] - 38s 32ms/step - loss: 0.2798 - accuracy: 0.9153 - val_loss: 0.1054 - val_accuracy: 0.9630
Epoch 4/10
1187/1187 [==============================] - 39s 33ms/step - loss: 0.2232 - accuracy: 0.9316 - val_loss: 0.0918 - val_accuracy: 0.9710
Epoch 5/10
1187/1187 [==============================] - 38s 32ms/step - loss: 0.1898 - accuracy: 0.9427 - val_loss: 0.0796 - val_accuracy: 0.9730
Epoch 6/10
1187/1187 [==============================] - 38s 32ms/step - loss: 0.1735 - accuracy: 0.9466 - val_loss: 0.0743 - val_accuracy: 0.9720
Epoch 7/10
1187/1187 [==============================] - 38s 32ms/step - loss: 0.1532 - accuracy: 0.9541 - val_loss: 0.0632 - val_accura